In [286]:
import pandas as pd
import numpy as np
from random import random, seed
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from math import sqrt
from datetime import timedelta

In [82]:
# BACA DATA
df = pd.read_excel("dataset aluminium lme.xlsx")
df.head()

,Tanggal,LME Aluminium Cash,LME Aluminium 3 Months,LME Alumina 3 Months
0,2020-06-01,1551.5,1540.0,255.75
1,2020-06-02,1499.5,1529.0,255.75
2,2020-06-03,1526.0,1555.5,255.75
3,2020-06-04,1535.0,1560.5,255.75
4,2020-06-05,1553.5,1579.0,255.75


In [251]:
# PROSES DATA MENJADI DATA DENGAN TARGET. PREPROCESS DILAKUKAN DENGAN MEMISAH DATA
# PER 3 HARI. 
def preprocess(dm, column):
    threeDaysData = pd.DataFrame(columns= ["x1", "x2", "y"])
    for i in range(len(dm["LME Aluminium Cash"])):
        if i == 0:
            threeDaysData.loc[i, "x1"] = max(dm[column][i:(i+1)*3])
            threeDaysData.loc[i, "x2"] = min(dm[column][i:(i+1)*3])
            threeDaysData.loc[i, "y"] = dm[column][i+3]
        elif (i)+3 < len(dm[column]):
            threeDaysData.loc[i, "x1"] = max(dm[column][i:(i)+3])
            threeDaysData.loc[i, "x2"] = min(dm[column][i:(i)+3] )
            threeDaysData.loc[i, "y"] = dm[column][i+3]
    return threeDaysData

In [90]:
# NORMALISASI MIN-MAX
def minmax(x):
    data = x.copy(deep=True)
    minimum = {}
    maximum = {}
    for i in data.columns:
        minimum[i] = data[i].min()
        maximum[i] = data[i].max()
    for i in data.index:
        for j in data.columns:
            data.loc[i, j] = (data.loc[i,j] - minimum[j]) / (maximum[j] - minimum[j])
    return data

In [85]:
# FUNGSI HITUNG SIGMOID DAN DIFERENSIAL (TURUNAN) SIGMOID
def sigmoid(value):
    return 1 / (1 + np.exp(-value))

def differ_sigmoid(value):
    return sigmoid(value) * (1 - sigmoid(value))

In [86]:
# FUNGSI UNTUK PELATIHAN
def back_train(x, epoch, alpha, layer, max_mse):
    et = 0
    ep = 0
    weightArray = []
    outputLayer = pd.DataFrame(columns=[i for i in range(layer)])
    inputBias = []
    b_out = 0
    mse = 1
    while (epoch == -1 or ep <  epoch) and mse > max_mse:

        print("MSE: ", mse, "Epoch: ", ep)
        dataIn = x.iloc[et]
        if ep == 0:
            for i in range(2):
                weightArray.append(pd.DataFrame(index=[0]))
            for i in range(len(weightArray)):
                for j in range(layer):
                    weightArray[i].loc[0, j] = random()
            for i in range(layer):
                inputBias.append(random())
            for i in range(len(outputLayer.columns)):
                outputLayer.loc[0, i] = random()
            outputLayer.loc[0, "b"] = random()
            b_out = random()
        else:
            inputBias = inputBiasNew
            weightArray = newWeightArray
            outputLayer = newOutputLayer
        z_net = []
        for i in range(layer):
            z_net.append((inputBias[i] + dataIn["x1"] * weightArray[0].loc[0, i] + dataIn["x2"] * weightArray[1].loc[0, i]
        ))
        z_activate = []
        #         z_activate.loc[i, j] = 1 / (1+(2.71828183 ** -(z_net.loc[i, j])))
        for i in range((layer)):
            z_activate.append(sigmoid(z_net[i]))
        y_net = []
        y_temp = []

        for i in range((layer)):
            y_temp.append((z_activate[i] * outputLayer.loc[0, i]))
        y_net.append(np.sum(y_temp)+b_out)
        y_activate = []
        for i in y_net:
            y_activate.append(sigmoid(i))
        error = dataIn["y"] - y_activate[0]
        mse = (error ** 2)
        backOutputTarget = []
        for i in range(len(y_activate)):
            backOutputTarget.append((dataIn["y"] - 
                                     y_activate[i]) * differ_sigmoid(y_net[i]))
        backAlphaHidden = []
        for i in range(layer):
            backAlphaHidden.append(alpha * backOutputTarget[0] * z_activate[i])
        backAlphaHidden.append(alpha * backOutputTarget[0] * b_out)
        errorTarget = []
        for i in range(layer):
            errorTarget.append(backOutputTarget[0] * outputLayer.loc[0, i])
        errorHidden = []
        for i in range(layer):
            errorHidden.append(errorTarget[i] * differ_sigmoid(z_net[i]))
        deltaInputHidden = []
        deltaInputBias = []
        for i in range(2):
            deltaInputHidden.append(pd.DataFrame(index=[i for i in range(1)]))
        for i in range(len(deltaInputHidden)):
            for j in range(layer):
                deltaInputHidden[i].loc[0, j] = alpha * errorHidden[j] * dataIn[i]
        for i in range(layer):
            deltaInputBias.append(alpha * errorHidden[i] * inputBias[i])
        newWeightArray = []
        inputBiasNew = []
        for i in range(2):
            newWeightArray.append(pd.DataFrame(index=[i for i in range(1)]))
        for i in range(len(newWeightArray)):
            for j in range(layer):
                newWeightArray[i].loc[0, j] = deltaInputHidden[i].loc[0, j] + weightArray[i].loc[0, j]
        for i in range(layer):
            inputBiasNew.append(inputBias[i] - deltaInputBias[i])
        newOutputLayer = pd.DataFrame(index = [i for i in range(1)], columns=[i for i in range(layer)])
        for i in newOutputLayer.index:
            for j in newOutputLayer.columns:
                newOutputLayer.loc[i, j] = backAlphaHidden[j] + outputLayer.loc[0, j]
        for i in newOutputLayer.index:
            newOutputLayer.loc[i, "b"] = backAlphaHidden[len(backAlphaHidden)-1] + b_out
        
        ep += 1

        if et < len(dataIn):
            et += 1
        elif et > len(dataIn) - 1:
            et = 0
    return weightArray, outputLayer, inputBias

In [174]:
# PENGUJIAN
def back_test(x, weightArray, outputLayer, inputBias, layer):
    result = []
    for m in range(len(x)):
        dataIn = x.iloc[m]
        z_net = []
        for i in range(layer):
            z_net.append((inputBias[i] + dataIn["x1"] * weightArray[0].loc[0, i] + dataIn["x2"] * weightArray[1].loc[0, i]
        ))
        z_activate = []
        #         z_activate.loc[i, j] = 1 / (1+(2.71828183 ** -(z_net.loc[i, j])))
        for i in range(layer):
            z_activate.append(sigmoid(z_net[i]))
        y_net = []
        y_temp = []

        for i in range(layer):
            y_temp.append((z_activate[i] * outputLayer.loc[0, i]))
        y_net.append(np.sum(y_temp)+outputLayer.loc[0, "b"])
        y_activate = []
        for i in y_net:
            y_activate.append(sigmoid(i))
            result.append(sigmoid(i))
    return result

In [252]:
# PREPROCESS DAN NORMALISASI
col = []
for i in df.columns:
    if i != "Tanggal":
        col.append(i)
threeDaysData = preprocess(df, col[0]) #0 untuk LME AL. CASH, 1 LME AL. 3 MONTHS, 2 LME. ALUMINA
normalData = minmax(threeDaysData)
normalData

,x1,x2,y
0,0.108553,0,0
1,0,0,0.121711
2,0.121711,0.145604,0.121711
3,0.121711,0.195055,0.121711
4,0.121711,0.296703,0.226974
5,0.226974,0.296703,0.200658
6,0.226974,0.296703,0.516447
7,0.516447,0.362637,0.309211
8,0.516447,0.362637,0.164474
9,0.516447,0.332418,0.164474


In [256]:
# DATA COLUMN 0
# Best accuracy on this settings reached 87,8%
# seed(1)
# train = normalData.iloc[0:50]
# test = normalData.iloc[50:]
# layer = 5 #change to 3 its acc 90% to max 5 layer result 90.5%
# weightArray, outputLayer, inputBias = (back_train(train, 1, 0.1, layer, .1))

# Settings 2 -> Acc 86.9%
# seed(1)
# train = normalData.iloc[0:50]
# test = normalData.iloc[50:]
# layer = 12
# weightArray, outputLayer, inputBias = (back_train(train, 10, 0.1, layer, .1))

# seed(42)
# train = normalData.iloc[0:50]
# test = normalData.iloc[50:]
# layer = 20
# weightArray, outputLayer, inputBias = (back_train(train, 1000, 0.1, layer, .1))

#PELATIHAN
seed(42)
train = normalData.iloc[0:50]
test = normalData.iloc[50:]
layer = 5
weightArray, outputLayer, inputBias = (back_train(train, 50, 0.1, layer, .1))

MSE:  1 Epoch:  0
MSE:  0.7575388186523417 Epoch:  1
MSE:  0.5544235102976064 Epoch:  2
MSE:  0.5610691381732162 Epoch:  3
MSE:  0.5603088834365874 Epoch:  4
MSE:  0.7451596968755925 Epoch:  5
MSE:  0.543639131587475 Epoch:  6
MSE:  0.5499347893222253 Epoch:  7
MSE:  0.5489439538304332 Epoch:  8
MSE:  0.7318160498906064 Epoch:  9
MSE:  0.5320429965057526 Epoch:  10
MSE:  0.5379315252495361 Epoch:  11
MSE:  0.5366868662393217 Epoch:  12
MSE:  0.7174374767830718 Epoch:  13
MSE:  0.5195815361144012 Epoch:  14
MSE:  0.5250005257355088 Epoch:  15
MSE:  0.5234783248729381 Epoch:  16
MSE:  0.701955846320338 Epoch:  17
MSE:  0.5062048403080496 Epoch:  18
MSE:  0.5110871101547425 Epoch:  19
MSE:  0.5092639299716746 Epoch:  20
MSE:  0.6853086207147611 Epoch:  21
MSE:  0.4918697802747552 Epoch:  22
MSE:  0.49614429962117446 Epoch:  23
MSE:  0.4939980222716923 Epoch:  24
MSE:  0.6674431783713262 Epoch:  25
MSE:  0.47654395374663566 Epoch:  26
MSE:  0.48013731654568653 Epoch:  27
MSE:  0.4776484944

In [257]:
#PENGUJIAN
hasil = back_test(test, weightArray, outputLayer, inputBias, layer)
hasil

[0.738532198911448,
 0.7388158898276896,
 0.7390357384611237,
 0.7388275201386593,
 0.739051843205029,
 0.7393602107741449,
 0.7403103836563588,
 0.7408768287306198,
 0.7414968316781139]

In [258]:
#EVALUASI

#MAE
mae = mean_absolute_error(test["y"], hasil)
print('MAE: %f' % mae)

#MSE
mse = mean_squared_error(test["y"], hasil)
print('MSE: %f' % mse)

#RMSE
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#MAPE (Mean Absolute Percentage Error)
actual = np.array([test["y"]])
forecast = np.array([hasil])
mape=np.mean(np.abs((actual - forecast) / actual)) * 100
print('MAPE: %f' % mape)

# ACCURACY
Accuracy= 100-mape
print('Accuracy: %f' % Accuracy)

MAE: 0.152589
MSE: 0.031282
RMSE: 0.176866
MAPE: 16.249575
Accuracy: 83.750425


In [259]:
#PENGUBAHAN HASIL PENGUJIAN MENJADI DATA ANGKA (DENORMALISASI MIN-MAX)
def denormalisasi(data, predict):
    for i in predict:
        return i * (max(data) - min(data)) + min(data)

print(denormalisasi(threeDaysData["y"], forecast))
print(threeDaysData["y"][50:])


[1647.25689423 1647.30001525 1647.33343225 1647.30178306 1647.33588017
 1647.38275204 1647.52717832 1647.61327797 1647.70751842]
50    1654.5
51    1654.5
52    1654.5
53    1661.5
54      1671
55      1687
56    1681.5
57    1685.5
58    1685.5
Name: y, dtype: object


In [13]:
# DATA COLUMN 2
# seed(1)
# train = normalData.iloc[0:50]
# test = normalData.iloc[50:]
# layer = 2
# weightArray, outputLayer, inputBias = (back_train(train, 2, 0.1, layer, .1))

# seed(42)
# train = normalData.iloc[0:50]
# test = normalData.iloc[50:]
# layer = 20
# weightArray, outputLayer, inputBias = (back_train(train, 100, 0.1, layer, .1))

# DATA COLUMN 3
# seed(1)
# train = normalData.iloc[0:50]
# test = normalData.iloc[50:]
# layer = 2
# weightArray, outputLayer, inputBias = (back_train(train, 100, 0.1, layer, .1))

#### Kesimpulannya, untuk setiap kolom yang ingin diujikan prediksinya, gunakan setting yang berbeda beda saat pelatihan dan pengujian. Karena setiap kolom memiliki setting parameter tersendiri yang hasil akurasinya akan lebih bagus

# Prediksi Data Selanjutnya

In [154]:
col = []
for i in df.columns:
    if i != "Tanggal":
        col.append(i)

alCash = minmax(preprocess(df, col[0]))
alThree = minmax(preprocess(df, col[1]))
alumina = minmax(preprocess(df, col[2]))

seed(1)
L0 = 5 #change to 3 its acc 90% to max 5 layer result 90.5%
w0, o0, i0 = (back_train(alCash, 1, 0.1, L0, .1))

MSE:  1 Epoch:  0


In [16]:
seed(1)
L1 = 2
w1, o1, i1 = (back_train(alThree, 2, 0.1, L1, .1))

MSE:  1 Epoch:  0
MSE:  0.5097601411859541 Epoch:  1


In [17]:
seed(1)
L2 = 2
w2, o2, i2 = (back_train(alumina, 100, 0.1, L2, .1))

MSE:  1 Epoch:  0
MSE:  0.5097601411859541 Epoch:  1
MSE:  0.5066466118294598 Epoch:  2
MSE:  0.5035184075193765 Epoch:  3
MSE:  0.5003759260129172 Epoch:  4
MSE:  0.49721957911404413 Epoch:  5
MSE:  0.49404979256993936 Epoch:  6
MSE:  0.4908670059416905 Epoch:  7
MSE:  0.4876716724486705 Epoch:  8
MSE:  0.48446425878614996 Epoch:  9
MSE:  0.4812452449157703 Epoch:  10
MSE:  0.47801512382856715 Epoch:  11
MSE:  0.47477440128033566 Epoch:  12
MSE:  0.47152359549919953 Epoch:  13
MSE:  0.46826323686535215 Epoch:  14
MSE:  0.46499386756302613 Epoch:  15
MSE:  0.46171604120484844 Epoch:  16
MSE:  0.4584303224288388 Epoch:  17
MSE:  0.45513728646841484 Epoch:  18
MSE:  0.4518375186958724 Epoch:  19
MSE:  0.44853161413991416 Epoch:  20
MSE:  0.44522017697790855 Epoch:  21
MSE:  0.44190382000366857 Epoch:  22
MSE:  0.4385831640716454 Epoch:  23
MSE:  0.4352588375185364 Epoch:  24
MSE:  0.4319314755634098 Epoch:  25
MSE:  0.42860171968755184 Epoch:  26
MSE:  0.425270216995333 Epoch:  27
MSE:  

In [295]:
def predict(iterate):
    df_new = df.copy(deep=True)

    for i in range(iterate):
        predictTable0 = pd.DataFrame(columns=["x1", "x2", "y"])
        predictTable1 = pd.DataFrame(columns=["x1", "x2", "y"])
        predictTable2 = pd.DataFrame(columns=["x1", "x2", "y"])

        predictTable0.loc[i, "x1"] = max(df_new.loc[len(df_new)-3:len(df_new), "LME Aluminium Cash"] )
        predictTable0.loc[i, "x2"] = min(df_new.loc[len(df_new)-3:len(df_new), "LME Aluminium Cash"] )

        predictTable0.loc[i, "x1"] = (predictTable0.loc[i, "x1"] - min(threeDaysData["x1"])) / (max (threeDaysData["x1"]) - min(threeDaysData["x1"]))
        predictTable0.loc[i, "x2"] = (predictTable0.loc[i, "x2"] - min(threeDaysData["x2"])) / (max (threeDaysData["x2"]) - min(threeDaysData["x2"]))

        predictTable0.loc[i, "y"] = back_test(pd.DataFrame(predictTable0.loc[i, :]).transpose(), w0, o0, i0, L0)[0]

        predictTable1 = pd.DataFrame(columns=["x1", "x2", "y"])
        predictTable1.loc[i, "x1"] = max(df_new.loc[len(df_new)-3:len(df_new), "LME Aluminium 3 Months"] )
        predictTable1.loc[i, "x2"] = min(df_new.loc[len(df_new)-3:len(df_new), "LME Aluminium 3 Months"] )

        predictTable1.loc[i, "x1"] = (predictTable1.loc[i, "x1"] - min(threeDaysData["x1"])) / (max (threeDaysData["x1"]) - min(threeDaysData["x1"]))
        predictTable1.loc[i, "x2"] = (predictTable1.loc[i, "x2"] - min(threeDaysData["x2"])) / (max (threeDaysData["x2"]) - min(threeDaysData["x2"]))

        predictTable1.loc[i, "y"] = back_test(pd.DataFrame(predictTable1.loc[i, :]).transpose(), w1, o1, i1, L1)[0]

        
        predictTable2 = pd.DataFrame(columns=["x1", "x2", "y"])
        predictTable2.loc[i, "x1"] = max(df_new.loc[len(df_new)-3:len(df_new), "LME Alumina 3 Months"] )
        predictTable2.loc[i, "x2"] = min(df_new.loc[len(df_new)-3:len(df_new), "LME Alumina 3 Months"] )

        predictTable2.loc[i, "x1"] = (predictTable2.loc[i, "x1"] - min(threeDaysData["x1"])) / (max (threeDaysData["x1"]) - min(threeDaysData["x1"]))
        predictTable2.loc[i, "x2"] = (predictTable2.loc[i, "x2"] - min(threeDaysData["x2"])) / (max (threeDaysData["x2"]) - min(threeDaysData["x2"]))

        predictTable2.loc[i, "y"] = back_test(pd.DataFrame(predictTable2.loc[i, :]).transpose(), w2, o2, i2, L2)[0]
        

        df_new.loc[len(df_new), "LME Aluminium Cash"] = denormalisasi(preprocess(df, col[0])["y"], [predictTable0.loc[i, "y"]])
        df_new.loc[len(df_new)-1, "LME Aluminium 3 Months"] = denormalisasi(preprocess(df, col[1])["y"], [predictTable1.loc[i, "y"]])
        df_new.loc[len(df_new)-1, "LME Alumina 3 Months"] = denormalisasi(preprocess(df, col[2])["y"], [predictTable2.loc[i, "y"]])
        df_new.loc[len(df_new)-1, "Tanggal"] = df_new.loc[len(df_new)-2, "Tanggal"] + timedelta(days=1)

    return df_new.round(1)
df_new = predict(10)
df_new

,Tanggal,LME Aluminium Cash,LME Aluminium 3 Months,LME Alumina 3 Months
0,2020-06-01,1551.5,1540.0,255.8
1,2020-06-02,1499.5,1529.0,255.8
2,2020-06-03,1526.0,1555.5,255.8
3,2020-06-04,1535.0,1560.5,255.8
4,2020-06-05,1553.5,1579.0,255.8
...,...,...,...,...
67,2020-08-07,1671.8,1687.8,265.5
68,2020-08-08,1671.8,1687.8,265.5
69,2020-08-09,1671.8,1687.8,265.5
70,2020-08-10,1671.8,1687.8,265.5


#### Prediksi di atas memiliki hasil yang kurang bervariasi pada kolom LME Aluminium Cash dan LME Aluminium 3 Months. Hal ini dikarenakan pada data latih (seluruh data), perubahan nilai perhari pada dua kolom ini selalu terjadi dan kenaikan serta penurunannya terjadi secara fluktuatif, sehingga prediksinya menjadi jelek dan kurang bervariasi seperti pada data latih. Berbeda dengan LME Alumina 3 Months yang perubahan nilainya cenderung tidak terjadi secara terus menerus, sehingga hasilnya lebih memuaskan.